**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.**



In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sympy.utilities.lambdify import lambdify
import sympy as sp

## Gradient Descent

This notebook illustrates the basic principles of gradient descent. Let us recall that gradient descent is an gradient-based iterative optimization method. The optimization starts from a certain initial point $\mathbf{x}_0$, which is at each step being shifted a little against the direction of the gradient. Since gradient represents the direction of the steepest ascent of a function at the specified point, by going against the direction of the gradient the function will be minimized.

The update rule which is applied at every step to compute the next point $\mathbf{x}_{i+1}$ is as follows:

\begin{equation}
\mathbf{x}_{i+1} = \mathbf{x}_i - \gamma \nabla f(\mathbf{x}_i)
\end{equation}
where $\nabla f(\mathbf{x}_i)$ is the gradient of the minimized function and $\gamma$, usually a small number from interval $( 0, 1 \rangle$, is the learning rate.

### Minimization of a Paraboloid: An Example

As an example that illustrates gradient descent we are going to use the minimization of a simple function – a paraboloid according to:

\begin{equation}
z = f(x, y) = 2x^2 + y^2
\end{equation}
#### Visualization of the Paraboloid

As the first step we are going to define and visualize the function:



In [ ]:
def f(x, y):
    return 2*x**2 + y**2

We are going to generate all combinations of points $x, y$ from a certain range and compute $z = f(x, y)$ for them:



In [ ]:
xx, yy = np.mgrid[-10:10.2:0.2, -10:10.2:0.2]
zz = f(xx, yy)

We are going to visualize the results in a 3D plot:



In [ ]:
plt.figure()
# ax = plt.gca(projection='3d')
ax = plt.subplot(projection='3d')
ax.plot_surface(xx, yy, zz, cmap='Spectral',
                linewidth=0, antialiased=True)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

3D plots are notorious for being hard to read because some of their elements tend to overlap with each other. To make our plots more readable we will therefore use 2D contour plots instead of 3D plots:



In [ ]:
plt.figure()
plt.contour(xx, yy, zz, cmap='Spectral')
# both axes at the same scale + create a legend
plt.gca().set_aspect('equal')
plt.xlabel('x'); plt.ylabel('y')
plt.colorbar(label='z')

Let's wrap this code in an auxiliary function so that we need not repeat it each time from now on:



In [ ]:
def plot_func(xx, yy, f, X=None):
    if not X is None:
        Xmin, Xmax = X[:, 0].min(), X[:, 0].max()
        Ymin, Ymax = X[:, 1].min(), X[:, 1].max()

        if (Xmin < xx.min() or Xmax > xx.max() or
                Ymin < yy.min() or Ymax > yy.max()):
            xx = np.linspace(Xmin, Xmax, 100)
            yy = np.linspace(Ymin, Ymax, 100)
            xx, yy = np.meshgrid(xx, yy)

        plt.scatter(X[:, 0], X[:, 1], zorder=10)

    zz = f(xx, yy)
    plt.contour(xx, yy, zz, cmap='Spectral')
    # both axes at the same scale + create a legend
    plt.gca().set_aspect('equal')
    plt.xlabel('x'); plt.ylabel('y')
    plt.colorbar(label='z')

### Gradient of a Function

To be able to minimize function $f(x, y)$ using gradient descent, we need to determine its gradient. Let us recall that gradient $\nabla f(x, y)$ of function $f(x, y)$ is the vector of its first-order partial derivatives. For a two dimensional function $f(x,y)$:

\begin{equation}
\nabla f(x, y) = \left(
    \frac{\partial f}{\partial x},
    \frac{\partial f}{\partial y}
\right)^T.
\end{equation}
Let us also recall that our function has the form of $f(x, y) = 2x^2 + y^2$. It is therefore easy to determine the partial derivatives. When computing the partial derivative for $x$, term $y^2$ will be considered a constant and we will only be differentiating $2x^2$. When computing the partial derivative for $y$ it will be vice versa. And so we obtain:

\begin{align}
\frac{\partial f}{\partial x} &= 4x \\[0.75em]
\frac{\partial f}{\partial y} &= 2y
\end{align}
Our iterative update rule was expressed in vector form – that is to say, for our function with two arguments $x, y$, vector $\mathbf{x}$ will be 2-dimensional and it will take the form of $\mathbf{x} = (x, y)^T$.

---
#### Task 1: Computing the Gradient

**Fill in the blanks in the following cell so that function `grad_f` will return the gradient of function $f(x, y)$ as a vector:**

---


In [ ]:
def grad_f(x, y):
    #return the gradient here

### Visualizing the Gradient

As we know, the gradient of a function indicates the direction of its steepest ascent. To get a better idea of what this means we can visualize the gradient, which we have just defined:



In [ ]:
xxg, yyg = np.mgrid[-10:11:1.5, -10:11:1.5]
gg = np.array(
    [[grad_f(x, y) for x, y in zip(rx, ry)]
          for rx, ry in zip(xxg, yyg)]
)

In [ ]:
plt.figure(figsize=[8,8])
plot_func(xx, yy, f)
plt.quiver(xxg, yyg, gg[..., 0], gg[..., 1])

The arrows indicate the gradient's direction. As we can see, they are all pointing outwards – in the direction the paraboloid increases. The size of the arrows indicates the magnitude of the gradient. The arrows close to the centre are tiny (the derivative at a minimum is zero) and they grow towards the margins: because the function grows faster and faster.

### Gradient Descent

Now we will continue by applying gradient descent in order to minimize our function. As the first step we will define a few parameters: the number of steps and the learning rate:



In [ ]:
num_steps = 20
learning_rate = 0.1

We will store all the computed points in matrix $X$ so that we can later visualize them. The matrix will have 2 columns – first for $x$ and the second for $y$, so the rows of the matrix represent the points of the gradient descent.



In [ ]:
X = np.zeros((num_steps + 1, 2))

The initial point can either be selected randomly:



In [ ]:
X[0] = np.random.uniform(-10, 10, (2,))

or we can opt in for some fixed point, so that we get the same result every time:



In [ ]:
X[0] = [-9, -8]

Let us once again recall our iterative update rule for computing the next point:
\begin{equation}
\mathbf{x}_{i+1} = \mathbf{x}_i - \gamma \nabla f(\mathbf{x}_i)
\end{equation}
where $\gamma$ is the learning rate and $\nabla f(\mathbf{x}_i)$ is the gradient of the function we are going to minimize.

---
#### Task 2: Implementing Gradient Descent

**Complete the loop below, so it will iteratively compute the gradient for the given number of steps. Add each point to the matrix $X$:**

---
Hint: the first row of the matrix contains the initial point, the second row will containt the point after the gradient step etc.

In [ ]:
for i in range(num_steps):
    # add your code here

The computed points can finally be visualized and we can ascertain that they really do converge to the minimum of the paraboloid:



In [ ]:
plot_func(xx, yy, f, X)

---
#### Task 3: Wrapping the Code in a Function

Let us now again wrap the codes above for gradient descent into a function so that we can call it repeatedly. Notice, that the function takes a parameter `grad_func`, where you can pass another function that returns the gradient. If you never used this feature of Python please check [this link ](https://www.geeksforgeeks.org/passing-function-as-an-argument-in-python/). Also notice that we use the unpacking operator `*` on `X[i]` to pass each its element as a separate arguemnt to the `grad_func`.

---


In [ ]:
def grad_desc(grad_func, init_point,
              num_steps, learning_rate):
    X = np.zeros((num_steps + 1, 2))
    # add the initial point
    # add the for loop
    return X

We can test our new function (notice how we pass the `grad_f`), display the gradient and print the minima using:



In [ ]:
X = grad_desc(grad_f, init_point=[-9, -8],
              num_steps=20, learning_rate=0.1)
plot_func(xx, yy, f, X)
print(f"Found minima at the point: {X[-1]}")

### Testing Different Learning Rates

To illustrate how things work we will now try doing gradient descent with different learning rates.

Let's start with $\gamma = 0.45$:



In [ ]:
X = grad_desc(grad_f, init_point=[-9, -8], num_steps=20,
      learning_rate=0.45
)
plot_func(xx, yy, f, X)

print(f"Found minima at the point: {X[-1]}")

In the plot above you can notice, that the learning rate is quite high, and therefore the $\mathbf{x}_i$ "jumps" between positive and negative values of x axis till it reaches some minima.

For $\gamma = 0.5$ the algorithm will start to oscillate and it will no longer converge to the minimum:



In [ ]:
X = grad_desc(grad_f, init_point=[-8,-4], num_steps=2,
      learning_rate=0.5
)
plot_func(xx, yy, f, X)
print(f"Found minima at the point: {X[-1]}")

With $\gamma > 0.5$ the algorithm will start to diverge and it will actually keep moving away from the minimum (For a better visualisation we changed the coordinates of the intial point to $[-1, 1]$):



In [ ]:
X = grad_desc(grad_f, init_point=[1, -1], num_steps=10,
      learning_rate=0.528
)
plot_func(xx, yy, f, X)

### Is the gradient really steepest ascent?
If you look at the contour plot and the function $f(x,y)$, you may notice that the function looks like increasing faster just in the direction of the $x$ axis. How can we verify that the function increases faster in the direction of gradient than in just the direction of the $x$ axis?

We can compare the increase in the direction of gradient to the increase in the direction of $x$ axis for a vector of lenght 1.

As a direction for the $x$ axis we can a vector $\mathbf{a}$ of length 1, i.e., $\rVert\mathbf{a}\lVert_2=1$, which points in the direction of the x axis (you will need to find such vector).
We will use a point $v=[1,1]$ as our starting point. Next, we need to define vector $\mathbf{g}$ that is created by rescaling $\nabla f(\mathbf{v})$ to an unit length, so we move from the point the same step size as with $a$. Then we compare if $f(\mathbf{v} + \mathbf{g}) > f(\mathbf{v} + \mathbf{a})$, i.e., if gradients sends us from point $\mathbf{v}$ to a larger value of $f$ than the $x$ axis direction $\mathbf{a}$ for the same step size.

---
#### Task 4: Verifying steepest ascent
Compute $\mathbf{g}$ (using vector norms), create vector $\mathbf{a}$ and then verify if $f(\mathbf{v} + \mathbf{g}) > f(\mathbf{v} + \mathbf{a})$.

---

In [ ]:
g = np.array([0, 0])
a = np.array([0, 0])
v = np.array([1, 1])
print(f"Value of f for the step in gradient direction {f(*(v + g))}")
print(f"Value of f for the step in direction of x axis {a}")

## Improving the gradient function

Below, we have a bit improved gradient descent function.

---
#### Task 5 (non-mandatory) - stopping criterion

---

To avoid iterating forever, we can add a stopping criterion to the gradient function. The stopping criterion will break the for loop, if the all the elements of the difference between the updated and the previous step $|\mathbf{x}_{i+1} - \mathbf{x}_i|$ are smaller than the tolerance criterion, passed by the `relative_tolerance` parameter. Also, keep only the rows of `X`, that were  reached by the for loop, so we omit the -1's.

In [ ]:
def grad_desc_i(grad_func, init_point, num_steps, learning_rate, relative_tolerance=0):
    # generalization of X for multiple dimensions
    X = np.zeros((num_steps + 1, len(init_point))) - 1
    X[0] = init_point

    for i in range(num_steps):
        diff = learning_rate * grad_func(X[i])
        X[i+1] = X[i] - diff
        # check the tolerance here
        # truncate the X vector and break the loop

    return X

If you have noticed, we pass a vector directly to the `grad_func` without unzipping it. Therefore, we will create new gradient function `grad_fv` that will have a vector as an argument.

---
#### Task 6 - gradient function with vector as parameter
Modify the `grad_v` function, so it takes vector as an argument and returns the value of gradient for the provided vector, e.g., instead of `y` you can use `x[1]`.

---

In [ ]:
def grad_fv(x):
    # return the vector

Now, we can check the output of the improved gradient descent function.

In [ ]:
X = grad_desc_i(grad_fv, init_point=[8, -9], num_steps=100,
                learning_rate=0.01, relative_tolerance=0.05)
print(f"minima at point: {X[-1]}")
X.shape

## A Multidimensional example for gradient descent
Below we have a multidimensional polynomial function. Now, we have a more complicated function $f(x,y,z) = x^2 + y^2 + 3z^2 + xy + 9x - 10y + 20z + 5$. Note, that we use a vector to pass the values of $x, y, z$ arguments.

In [ ]:
def cost_fun(x):
  return x[0]**2 + x[1]**2 + 3*x[2]**2 + x[0]*x[1] + 9*x[0] - 10*x[1] + 20*x[2] + 5

---
#### Task 7 - computing gradient of a function

---

Try to compute its gradient on a paper and then modify the a function `grad_fyx` so it will return the gradient.

In [ ]:
def grad_xyz(x):
  return np.array([
      x[0],
      x[1],
      x[2]])

Below, we run the gradient descent algorithm passing it the gradient function defined above. Print the point at which we found the minima as well as the function value corresponding to the point.

In [ ]:
init_point = np.array([20, -11, 5])
X = grad_desc_i(grad_xyz, init_point, 100, 0.005)
print(f"Final point: {np.round(X[-1],3)}, function value: {cost_fun(X[-1]):.4f}")

Now, you will need to find the minima of the function, but first we need to keep track of the gradient descent's progress. As the minimized function has more dimensions than we can fit to a two dimensional plot, you can print some values of the function's output matrix and corresponding function values to tune the learning rate.

---
#### Task 8 - printing the progress of the gradient function

Complete the `print_grad_progress` function, which prints current point and function value of every `n_skip`-th step of the gradient descent. Also, you can round up the numbers to get a more readable print.

---

In [ ]:
def print_grad_progress(X, n_skip, cost_function):
    for i in range(1, X.shape[0], n_skip):
      # add the formatted print

In [ ]:
print_grad_progress(X, 10, cost_fun)

---
#### Task 9 - finding the minimum of the function

Use the `grad_desc_i` and `print_grad_progress` functions to find the minima of the function $f(x,y,x)$ by chagning the learning rate and number of iterations, so you will find the smallest value. What is the value of the minima? What are its coordinates?  

---

In [ ]:
# Find the best value and print the iterations!

## BONUS - We do not need to know derrivations, we do not need no thoughs control... Computing Symbolic Gradient Automatically

When deriving the gradient of our function above we have computed it analytically and then rewrote it as source code by hand. However, in Python it is possible to compute the symbolic gradient automatically – using the `sympy` package. In the following example we will show how this can be done.

We will start by defining some symbolic variables that we are going to need – $x$ and $y$ in our case:



In [ ]:
symx, symy = sp.symbols('x y')

We will now define function $f(x, y)$ using the symbolic variables:



In [ ]:
symf = 2*symx**2 + symy**2
symf

When computing the symbolic gradient we will use a little trick. We will first transform our scalar function into matrix form and then compute its Jacobian. We will get a row vector that corresponds to the gradient as a result:



In [ ]:
sym_grad_f = sp.Matrix([symf]).jacobian([symx, symy])
sym_grad_f

To be able to use the resulting symbolic representation of gradient to actually compute it for particular values, we'll need to convert it to a standard numeric function. We will also do the same for function $f$:



In [ ]:
f = lambdify((symx, symy), symf, "numpy")
grad_f_sim = lambdify((symx, symy), sym_grad_f, "numpy")

We can now apply gradient descent in exactly the same way we did before – but now we no longer need to compute the gradient by hand:



In [ ]:
xx, yy = np.mgrid[-10:10.2:0.2, -10:10.2:0.2]
zz = f(xx, yy)

X = grad_desc(grad_f_sim, init_point=[-9, -8],
              num_steps=20, learning_rate=0.1)
plot_func(xx, yy, f, X)